In [2]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Step 1: Clone the Sign Language MNIST dataset from GitHub
!git clone https://github.com/ardamavi/Sign-Language-Digits-Dataset.git

# Step 2: Load and preprocess the dataset
def load_data(dataset_path):
    images = []
    labels = []
    for folder in os.listdir(dataset_path):
        label = int(folder)  # Assuming folders are named 0, 1, ..., 9
        folder_path = os.path.join(dataset_path, folder)
        for filename in os.listdir(folder_path):
            image_path = os.path.join(folder_path, filename)
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            image = cv2.resize(image, (64, 64))
            images.append(image)
            labels.append(label)
    images = np.array(images, dtype=np.float32) / 255.0  # Normalize pixel values to [0, 1]
    labels = np.array(labels, dtype=np.int32)
    return images, labels

dataset_path = 'Sign-Language-Digits-Dataset/Dataset'
X, y = load_data(dataset_path)

# Step 3: Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 4: Build the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')  # 10 classes (0-9)
])

# Step 5: Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Step 6: Print model summary
model.summary()

# Step 7: Reshape X_train and X_test for grayscale images
X_train = X_train.reshape((-1, 64, 64, 1))
X_test = X_test.reshape((-1, 64, 64, 1))

# Step 8: Train the model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Step 9: Evaluate the model on the test set
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Step 10: Calculate accuracy
accuracy = accuracy_score(y_test, y_pred_classes)
print(f"Test Accuracy: {accuracy:.4f}")

# Step 11: Generate confusion matrix and classification report
conf_matrix = confusion_matrix(y_test, y_pred_classes)
class_report = classification_report(y_test, y_pred_classes)

print(f"Confusion Matrix:\n{conf_matrix}")
print(f"Classification Report:\n{class_report}")


Cloning into 'Sign-Language-Digits-Dataset'...
remote: Enumerating objects: 2095, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 2095 (delta 2), reused 0 (delta 0), pack-reused 2089
Receiving objects: 100% (2095/2095), 15.07 MiB | 11.59 MiB/s, done.
Resolving deltas: 100% (660/660), done.
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 31, 31, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (No

In [4]:

# Step 12: Predict on a new image
def predict_image(image_path):
    # Load and preprocess the image
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (64, 64))
    img = np.expand_dims(img, axis=-1)
    img = np.expand_dims(img, axis=0) / 255.0

    # Predict class probabilities
    pred = model.predict(img)
    class_idx = np.argmax(pred)

    # Map class index to sign language digit (0-9)
    sign_digits = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
    predicted_digit = sign_digits[class_idx]

    return predicted_digit

# Example usage:
image_path = 'sign2.jpg'  # Replace with your image path
predicted_digit = predict_image(image_path)
print(f"Predicted Sign Language Digit: {predicted_digit}")

1/1 [==============================] - 0s 38ms/step
Predicted Sign Language Digit: 8
